In [43]:
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import CountVectorizer
import sklearn.linear_model
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, roc_auc_score
import matplotlib.pyplot as plt
import pickle


In [44]:
import torch
from torch import nn
from torch.utils.data import DataLoader, Dataset, TensorDataset, random_split
from transformers import BertTokenizer, BertModel, AdamW, get_linear_schedule_with_warmup
from transformers import BertForSequenceClassification
from p2_estimator import p2_estimator


In [45]:
# overview the training data
x_train_df = pd.read_csv(os.path.join('data_reviews', 'x_train.csv'))
y_train_df = pd.read_csv(os.path.join('data_reviews', 'y_train.csv'))

tr_text_list = x_train_df.values.tolist()
tr_y_list = y_train_df.values.tolist()

tr_y = np.hstack(np.array(tr_y_list))

reviews_list = [val[1].lower() for val in tr_text_list]


In [46]:
# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)
# print(model.config)

In [47]:
# config["hidden_size"] = 64/128/32/...
# config["num_heads"] = 64/128/32/...
# config["hidden_size"] = 64/128/32/...

param_grid = { 'hidden_size' : [16, 32, 64, 128],
    'num_hidden_layers': [1, 3, 5, 7],
}

In [48]:
bert = p2_estimator()

auroc_scorer = make_scorer(roc_auc_score, needs_proba=True, greater_is_better=True)
grid_search = GridSearchCV(bert, param_grid, scoring=auroc_scorer, cv=5, refit=True, return_train_score=True)

grid_search.fit(reviews_list, tr_y)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


TypeError: 'BertConfig' object does not support item assignment